# Exercice 2

Dans cet exercice on va approximer une fonction à l'aide d'un réseau de neurone et d'une optimisation par algorithme génétique.

On commence par réaliser les imports nécessaires.

In [2]:
import numpy as np
from enum import Enum
import pandas as pd
from typing import Tuple

1. Créer la fonction d'erreur qui a une matrice de poids $ Q  = 
 \begin{pmatrix}
q_1 \\
q_2 \\
q_3 \\
\end{pmatrix} $ 
et une fonction d'activation $h$ renvoie l'erreur err($Q$).


In [3]:
input_size = 3
input_dim = 4
output_size = 1
output_dim = 4

class ActivationFunction(Enum):
    SIGMOID = "sigmoid"
    HEAVISIDE = "heaviside"
    LINEAR = "linear"

def sigmoid(x: np.ndarray) -> np.ndarray:
    """Compute the sigmoid of a vector"""
    return 1 / (1 + np.exp(-x))

def heaviside(x: np.ndarray) -> np.ndarray:
    """Compute the heaviside function of a vector"""
    return np.where(x >= 0, 1, 0)

def linear(x: np.ndarray, T: float) -> np.ndarray:
    """Compute the linear function of a vector"""
    return np.clip(T*x, 0, 1)

def activation(x: np.ndarray, T: float, function: ActivationFunction) -> np.ndarray:
    """Compute the activation function of a vector"""
    if function == ActivationFunction.SIGMOID:
        return sigmoid(x)
    elif function == ActivationFunction.HEAVISIDE:
        return heaviside(x)
    elif function == ActivationFunction.LINEAR:
        return linear(x, T)
    else:
        raise ValueError("The function must be sigmoid, heaviside or linear")

def norm2(v: np.ndarray) -> float:
    """Compute the norme 2 of a vector"""
    return np.sqrt(np.sum(v**2))

def error(function: ActivationFunction, parameter: float, input: np.ndarray, weight: np.ndarray, output: np.ndarray) -> float:
    """Compute the error of the network for a given input and output"""
    if weight.shape != (input_size,):
        raise ValueError("The weight must be a 1D array")
    if input.shape != (input_size, input_dim):
        raise ValueError("The input must be a matrix of size 3x3")
    if output.shape != (output_dim,):
        raise ValueError("The output must be a 1D array")
    return norm2(v=activation(x=(np.dot(input.T, weight) - output), T=parameter, function=function))

weight: np.ndarray  = np.array([1, 0, 0])
input: np.ndarray = np.array([[0, 1, 1, 0], [0, 1, 0, 1], [1, 1, 1, 1]])
output: np.ndarray = np.array([0, 1, 1, 0])
function = ActivationFunction.SIGMOID
parameter: int = 1
print(error(function=function, parameter=parameter, input=input, weight=weight, output=output))


1.0


2. Créer une population de poids, c’est-à-dire une matrice $P = (Q_1, Q_2, · · · , Q_{taille pop.})$ et un vecteur d’erreur $err = (err_1, · · · err_{taille pop.})$ avec $taille$ $pop.$ entre 10 et 20.

In [4]:
population_size: int = 15
weight_population: np.ndarray = np.random.rand(population_size, input_size)
err_population: np.ndarray = np.zeros(population_size)

for i in range(population_size):
    err_population[i] = error(function=function, parameter=parameter, input=input, weight=weight_population[i], output=output)

df = pd.DataFrame(err_population, columns=["Error"])

# Style the DataFrame
def highlight_min(s):
    """
    Highlight the minimum value in a Series yellow.
    """
    is_min = s == s.min()
    return ["background-color: yellow" if v else "" for v in is_min]

styled_df = (df.style
             .set_caption("Error of the population")
             .set_table_styles([{"selector": "th, td", "props": [("font-size", "20px"), ("padding", "5px 200px")]},
                                {"selector": "caption", "props": [("font-size",  "50px"), ("font-weight", "bold")]}])
             .format("{:.4f}")  # Format error values to 4 decimal places
             .apply(highlight_min, axis=0)  # Highlight the minimum error value
             .set_properties(**{"text-align": "center"}))  # Center-align the text

display(styled_df)



,Error
0,1.2971
1,1.2095
2,1.2982
3,1.3202
4,1.1875
5,1.1451
6,1.0189
7,1.1502
8,1.1273
9,1.3257


3. Créer un algorithme génétique dont l’objectif est de minimiser l’erreur du réseau. Tester avec différentes fonctions d’activation et valeur de T.

In [5]:
def fitness(error: float) -> float:
    return 1 / (1 + error)

def selection(population: np.ndarray, fitnesses: np.ndarray, num_parents: int) -> np.ndarray:
    sorted_indices = np.argsort(fitnesses)[::-1]
    top_indices = sorted_indices[:num_parents]
    parents = population[top_indices]
    return parents

def crossover(parents: np.ndarray, offspring_size: Tuple[int, int]) -> np.ndarray:
    offspring = np.empty(offspring_size)
    crossover_point = np.uint8(offspring_size[1] / 2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

def mutation(offspring: np.ndarray, mutation_rate: float) -> np.ndarray:
    for ind in range(offspring.shape[0]):
        for idx in range(offspring.shape[1]):
            if np.random.rand() < mutation_rate:
                random_value = np.random.uniform(-1.0, 1.0)
                offspring[ind, idx] = offspring[ind, idx] + random_value
    return offspring

def genetic_algorithm(function: ActivationFunction, parameter: float, input: np.ndarray, output: np.ndarray,
                      population_size: int, num_generations: int, num_parents: int, mutation_rate: float) -> np.ndarray:

    # Initialize the population
    population = np.random.rand(population_size, input_size)

    for _ in range(num_generations):
        # Calculate the fitness of each individual
        fitnesses = np.array([fitness(error(function, parameter, input, individual, output)) for individual in population])

        # Select the best individuals for reproduction
        parents = selection(population, fitnesses, num_parents)

        # Perform crossover to create new offspring
        offspring = crossover(parents, offspring_size=(population_size - parents.shape[0], input_size))

        # Perform mutation on the offspring
        offspring_mutated = mutation(offspring, mutation_rate)

        # Replace the old population with the new one
        population[:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = offspring_mutated

    # Return the best individual from the final population
    best_individual_idx = np.argmax([fitness(error(function, parameter, input, individual, output)) for individual in population])
    return population[best_individual_idx]

# Test the genetic algorithm with different activation functions and parameter T
population_size: int = 15
num_generations: int = 100
num_parents: int = 5
mutation_rate: int = 0.1
input: np.ndarray = np.array([[0, 1, 1, 0], [0, 1, 0, 1], [1, 1, 1, 1]])
output: np.ndarray = np.array([0, 1, 1, 0])

df = pd.DataFrame(columns=["Activation function", "Parameter T", "Best weights", "Error"])

for func in ActivationFunction:
    for parameter in [0.5, 1, 2]:

        # Run the genetic algorithm
        best_weights = genetic_algorithm(
            function=func,
            parameter=parameter,
            input=input,
            output=output,
            population_size=population_size,
            num_generations=num_generations,
            num_parents=num_parents,
            mutation_rate=mutation_rate
        )

        # Calculate the error of the best individual
        best_error = error(function=func, parameter=parameter, input=input, weight=best_weights, output=output)

        # Create a new DataFrame with the current iteration's data
        new_row = pd.DataFrame({
            "Activation function": [func],
            "Parameter T": [parameter],
            "Best weights": [best_weights],
            "Error": [best_error]
        })

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

styled_df = (df.style
             .set_caption("Error of the population")
             .set_table_styles([{"selector": "th, td", "props": [("font-size", "20px"), ("padding", "5px 20px")]},
                                {"selector": "caption", "props": [("font-size",  "50px"), ("font-weight", "bold")]}])
             .format({"Error": "{:.4e}"})  # Format error values to 4 decimal places
             .apply(highlight_min, subset=["Error"])  # Highlight the minimum error value in the "Error" column
             .set_properties(**{"text-align": "center"}))  # Center-align the text

display(styled_df)



,Activation function,Parameter T,Best weights,Error
0,ActivationFunction.SIGMOID,0.500000,[ -9.93819029 -4.1938138 -15.13618342],2.6699e-07
1,ActivationFunction.SIGMOID,1.000000,[ -6.29058689 -2.67550817 -18.63846799],8.0620e-09
2,ActivationFunction.SIGMOID,2.000000,[ -9.00779514 -5.03502531 -16.95710126],4.3215e-08
3,ActivationFunction.HEAVISIDE,0.500000,[-3.055415 -2.91760896 -2.95189078],0.0000e+00
4,ActivationFunction.HEAVISIDE,1.000000,[ 0.55589101 1.339135 -2.38507641],0.0000e+00
5,ActivationFunction.HEAVISIDE,2.000000,[-0.74119335 -3.08655442 -2.19693046],0.0000e+00
6,ActivationFunction.LINEAR,0.500000,[-0.55469903 -1.77785074 -0.81640687],0.0000e+00
7,ActivationFunction.LINEAR,1.000000,[ 1.08307461 2.87252839 -5.24034465],0.0000e+00
8,ActivationFunction.LINEAR,2.000000,[ 0.90441067 -2.53017772 -0.27959006],0.0000e+00


4. Implanter le réseau associé à l’architecture. Essayer de faire de même. Qu’observe t’on ?

In [6]:
# Test the genetic algorithm with different activation functions and parameter T
population_size: int = 15
num_generations: int = 100
num_parents: int = 5
mutation_rate: int = 0.1
input: np.ndarray = np.array([[0, 1, 1, 0], [0, 1, 0, 1], [1, 1, 1, 1]])
output: np.ndarray = np.array([0, 0, 1, 1])

df = pd.DataFrame(columns=["Activation function", "Parameter T", "Best weights", "Error"])

for func in ActivationFunction:
    for parameter in [0.5, 1, 2]:

        # Run the genetic algorithm
        best_weights = genetic_algorithm(
            function=func,
            parameter=parameter,
            input=input,
            output=output,
            population_size=population_size,
            num_generations=num_generations,
            num_parents=num_parents,
            mutation_rate=mutation_rate
        )

        # Calculate the error of the best individual
        best_error = error(function=func, parameter=parameter, input=input, weight=best_weights, output=output)

        # Create a new DataFrame with the current iteration's data
        new_row = pd.DataFrame({
            "Activation function": [func],
            "Parameter T": [parameter],
            "Best weights": [best_weights],
            "Error": [best_error]
        })

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

styled_df = (df.style
             .set_caption("Error of the population")
             .set_table_styles([{"selector": "th, td", "props": [("font-size", "20px"), ("padding", "5px 20px")]},
                                {"selector": "caption", "props": [("font-size",  "50px"), ("font-weight", "bold")]}])
             .format({"Error": "{:.4e}"})  # Format error values to 4 decimal places
             .apply(highlight_min, subset=["Error"])  # Highlight the minimum error value in the "Error" column
             .set_properties(**{"text-align": "center"}))  # Center-align the text

display(styled_df)

,Activation function,Parameter T,Best weights,Error
0,ActivationFunction.SIGMOID,0.500000,[ -7.36434361 -6.43378121 -17.49573968],2.5217e-08
1,ActivationFunction.SIGMOID,1.000000,[ -6.27577244 -2.10546791 -17.69376523],2.0708e-08
2,ActivationFunction.SIGMOID,2.000000,[ -7.3529024 -4.85434391 -19.42896465],3.6485e-09
3,ActivationFunction.HEAVISIDE,0.500000,[-3.6340723 -1.99947317 -1.77105541],0.0000e+00
4,ActivationFunction.HEAVISIDE,1.000000,[-0.17338757 0.53533757 -0.68385617],0.0000e+00
5,ActivationFunction.HEAVISIDE,2.000000,[-0.63238911 -1.31548952 -0.39099749],0.0000e+00
6,ActivationFunction.LINEAR,0.500000,[-2.14718954 -0.30416203 -1.13832047],0.0000e+00
7,ActivationFunction.LINEAR,1.000000,[-1.90636126 0.96998927 -2.4826332 ],0.0000e+00
8,ActivationFunction.LINEAR,2.000000,[-0.92058368 -3.10898227 -0.85613287],0.0000e+00


5. On ajoute une couche cachée de quatre neurones, chercher des valeurs de poids P1 entre la couche d’entrée et la couche cachée et P2 entre la couche cachée et la couche de sortie. Optimiser les poids de manière à minimiser l’erreur du réseau.

In [12]:
# Test the genetic algorithm with different activation functions and parameter T
population_size: int = 15
num_generations: int = 100
num_parents: int = 5
mutation_rate: int = 0.1
input: np.ndarray = np.array([[0, 1, 1, 0], [0, 1, 0, 1], [1, 1, 1, 1]])
input_size = 3
hidden_layer_size = 4
output_layer_size = 1
output: np.ndarray = np.array([0, 0, 1, 1])

def error(function: ActivationFunction, parameter: float, input: np.ndarray, hidden_weights: np.ndarray, output_weights: np.ndarray, output: np.ndarray) -> float:
    """Compute the error of the network for a given input and output"""
    
    hidden_layer_output = activation(x=(np.dot(input.T, hidden_weights)), T=parameter, function=function)

    output_layer_output = activation(x=(np.dot(hidden_layer_output, output_weights)), T=parameter, function=function)

    return norm2(output_layer_output - output)

def selection(hidden_population: np.ndarray, output_population: np.ndarray, fitnesses: np.ndarray, num_parents: int) -> np.ndarray:
    sorted_indices = np.argsort(fitnesses)[::-1]
    top_indices = sorted_indices[:num_parents]
    hidden_parents = hidden_population[top_indices]
    output_parents = output_population[top_indices]
    return hidden_parents, output_parents

def crossover(hidden_parents: np.ndarray, output_parents: np.ndarray, hidden_offspring_size: Tuple[int, int], output_offspring_size: Tuple[int, int]) -> Tuple[np.ndarray, np.ndarray]:
    hidden_offspring = np.empty(hidden_offspring_size)
    output_offspring = np.empty(output_offspring_size)
    
    hidden_crossover_point = np.uint8(hidden_offspring_size[1] / 2)
    output_crossover_point = np.uint8(output_offspring_size[1] / 2)

    # Crossover for hidden layer weights
    for k in range(hidden_offspring_size[0]):
        parent1_idx = k % hidden_parents.shape[0]
        parent2_idx = (k + 1) % hidden_parents.shape[0]
        """
        Hidden parents shape: (5, 3, 4)
        Hidden offspring shape: (10, 3)
        """
        hidden_offspring[k, :hidden_crossover_point] = hidden_parents[parent1_idx, :hidden_crossover_point]
        hidden_offspring[k, hidden_crossover_point:] = hidden_parents[parent2_idx, hidden_crossover_point:]

    # Crossover for output layer weights
    for k in range(output_offspring_size[0]):
        parent1_idx = k % output_parents.shape[0]
        parent2_idx = (k + 1) % output_parents.shape[0]
        output_offspring[k, :output_crossover_point] = output_parents[parent1_idx, :output_crossover_point]
        output_offspring[k, output_crossover_point:] = output_parents[parent2_idx, output_crossover_point:]

    return hidden_offspring, output_offspring

def mutation(hidden_offspring: np.ndarray, output_offspring: np.ndarray, mutation_rate: float) -> Tuple[np.ndarray, np.ndarray]:
    # Mutation for hidden layer weights
    for ind in range(hidden_offspring.shape[0]):
        for idx in range(hidden_offspring.shape[1]):
            if np.random.rand() < mutation_rate:
                random_value = np.random.uniform(-1.0, 1.0)
                hidden_offspring[ind, idx] = hidden_offspring[ind, idx] + random_value

    # Mutation for output layer weights
    for ind in range(output_offspring.shape[0]):
        for idx in range(output_offspring.shape[1]):
            if np.random.rand() < mutation_rate:
                random_value = np.random.uniform(-1.0, 1.0)
                output_offspring[ind, idx] = output_offspring[ind, idx] + random_value

    return hidden_offspring, output_offspring

def genetic_algorithm(function: ActivationFunction, parameter: float, input: np.ndarray, output: np.ndarray,
                      population_size: int, num_generations: int, num_parents: int, mutation_rate: float) -> np.ndarray:

    # Initialize the population
    hidden_weight_population = np.random.rand(population_size, input_size, hidden_layer_size)
    output_weight_population = np.random.rand(population_size, hidden_layer_size, output_layer_size)

    for _ in range(num_generations):
        # Calculate the fitness of each individual
        fitnesses = np.array([fitness(error(function=function,
                                            parameter=parameter,
                                            input=input,
                                            hidden_weights=hidden_weight,
                                            output_weights=output_weight,
                                            output=output))
                                            for hidden_weight, output_weight in zip(hidden_weight_population, output_weight_population)])

        # Select the best individuals for reproduction
        hidden_parents, output_parents = selection(hidden_weight_population, output_weight_population, fitnesses, num_parents)

        # Perform crossover to create new offspring
        hidden_offspring, output_offspring = crossover(hidden_parents, output_parents,
                               hidden_offspring_size=(population_size - hidden_parents.shape[0], input_size, hidden_layer_size),
                               output_offspring_size=(population_size - output_parents.shape[0], hidden_layer_size, output_layer_size))

        # Perform mutation on the offspring
        hidden_offspring, output_offspring = mutation(hidden_offspring, output_offspring, mutation_rate)

        # Replace the old population with the new one
        hidden_weight_population[0:hidden_parents.shape[0], :] = hidden_parents
        hidden_weight_population[hidden_parents.shape[0]:, :] = hidden_offspring

        output_weight_population[0:output_parents.shape[0], :] = output_parents
        output_weight_population[output_parents.shape[0]:, :] = output_offspring

    # Return the best individual from the final population
    best_individual_idx = np.argmax([fitness(error(function=function,
                                            parameter=parameter,
                                            input=input,
                                            hidden_weights=hidden_weight,
                                            output_weights=output_weight,
                                            output=output))
                                    for hidden_weight, output_weight in zip(hidden_weight_population, output_weight_population)])
    
    return hidden_weight_population[best_individual_idx], output_weight_population[best_individual_idx]

df = pd.DataFrame(columns=["Activation function", "Parameter T", "Best weights", "Error"])

for func in ActivationFunction:
    for parameter in [0.5, 1, 2]:

        # Run the genetic algorithm
        hidden_best_weights, output_best_weights = genetic_algorithm(
            function=func,
            parameter=parameter,
            input=input,
            output=output,
            population_size=population_size,
            num_generations=num_generations,
            num_parents=num_parents,
            mutation_rate=mutation_rate
        )

        # Calculate the error of the best individual
        error(function=function, 
              parameter=parameter, 
              input=input, 
              hidden_weights=hidden_best_weights, 
              output_weights=output_best_weights, 
              output=output)

        # Create a new DataFrame with the current iteration's data
        new_row = pd.DataFrame({
            "Activation function": [func],
            "Parameter T": [parameter],
            "Best weights": [hidden_best_weights],
            "Error": [best_error]
        })

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

styled_df = (df.style
             .set_caption("Error of the population")
             .set_table_styles([{"selector": "th, td", "props": [("font-size", "20px"), ("padding", "5px 20px")]},
                                {"selector": "caption", "props": [("font-size",  "50px"), ("font-weight", "bold")]}])
             .format({"Error": "{:.4e}"})  # Format error values to 4 decimal places
             .apply(highlight_min, subset=["Error"])  # Highlight the minimum error value in the "Error" column
             .set_properties(**{"text-align": "center"}))  # Center-align the text

display(styled_df)

,Activation function,Parameter T,Best weights,Error
0,ActivationFunction.SIGMOID,0.500000,[[ 0.93707556 1.16695553 1.01413438 0.61174087] [-0.13069322 -0.55539107 -0.35823516 -0.4958267 ] [ 0.80473913 1.52736188 1.5423408 0.9961655 ]],0.0000e+00
1,ActivationFunction.SIGMOID,1.000000,[[ 0.8956445 1.18467974 1.60438887 1.1535831 ] [-0.36262887 -0.24272854 -0.15215943 0.17100642] [ 0.1229911 -0.16683744 0.54673792 -0.31595211]],0.0000e+00
2,ActivationFunction.SIGMOID,2.000000,[[ 0.42118459 0.30045014 0.2719228 0.35004568] [-0.0831351 -0.02927235 0.30685718 0.3842255 ] [ 0.96207525 0.53306929 0.39283043 0.71448538]],0.0000e+00
3,ActivationFunction.HEAVISIDE,0.500000,[[-5.06234252 -4.69454256 -4.50855564 -4.75378164] [ 1.55678506 1.45079195 1.8221246 1.72356185] [ 2.2836109 2.81536373 3.09091922 2.62474346]],0.0000e+00
4,ActivationFunction.HEAVISIDE,1.000000,[[-0.84843136 -0.7493597 -0.85536392 -0.20742721] [ 2.91314337 3.32012612 2.69727895 3.5741606 ] [ 0.37154636 0.0710125 0.22884882 0.01368625]],0.0000e+00
5,ActivationFunction.HEAVISIDE,2.000000,[[ 0.67364702 -0.02615319 0.85374139 0.7432377 ] [ 0.94620522 1.73841553 1.31775609 1.35397128] [ 0.37167955 0.79987108 1.3209278 0.98437633]],0.0000e+00
6,ActivationFunction.LINEAR,0.500000,[[ 0.40149255 0.13011641 -0.30321723 0.12602835] [ 0.62216782 0.75153855 1.16876352 1.10104185] [ 3.25814644 3.09450473 2.8307552 2.55646485]],0.0000e+00
7,ActivationFunction.LINEAR,1.000000,[[-0.65668746 -0.26655887 -0.45790521 -0.39762471] [ 1.52596206 1.57680507 1.42736864 1.89441706] [ 2.56623939 1.70333753 2.52539144 2.56691583]],0.0000e+00
8,ActivationFunction.LINEAR,2.000000,[[-0.13013555 -0.10306636 -0.56787907 0.24691496] [-0.15440538 -0.37946554 -0.38311286 -0.11750894] [ 1.2808465 1.48210678 1.35867437 1.29749533]],0.0000e+00
